In [6]:
#Import Libraries
import arcpy 
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [7]:
arcpy.env.workspace = 'G:\Proj1_SSCI575\Proj1_SSCI575\Data Wrangling.gdb'

## EDA

In [35]:
arcpy.ListFeatureClasses() #Feature Classes

['building_footprints', 'OK_earthquakes', 'OK_well_injection', 'school_sites', 'traffic_volume', 'census_tracts10', 'Medford_earthquakes', 'Medford_5mi', 'Medford_25mi', 'Medford_well_injection']

In [5]:
#Functoin to display column names
def display_fields(feature):
    fields_ls = []
    fields = arcpy.ListFields(feature)
    for f in fields: fields_ls.append(f.name)
    return fields_ls

display_fields('OK_earthquakes')

['OBJECTID', 'Shape', 'time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst', 'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'magNst', 'status', 'locationSource', 'magSource', 'updated_Converted', 'Date', 'Date_1', 'Month', 'Year', 'Earthquake', 'City']

In [4]:
display_fields('OK_well_injection')

OBJECTID  ﻿Shape  ﻿API  ﻿Well_Name  ﻿Well_Number  ﻿Operator_Number  ﻿Report_Date  ﻿Volume_BPD  ﻿Pressure  ﻿API_NUM  ﻿

In [14]:
#Function to display data types
def get_types(field):
    fields = arcpy.ListFields(field)
    for f in fields: print('{}: {}'.format(f.name, f.type))
        
get_types('OK_earthquakes')

OBJECTID: OID
Shape: Geometry
time: String
latitude: Double
longitude: Double
depth: Double
mag: Double
magType: String
nst: String
gap: Integer
dmin: Double
rms: Double
net: String
id: String
updated: String
place: String
type: String
horizontalError: Double
depthError: Double
magError: Double
magNst: Integer
status: String
locationSource: String
magSource: String
updated_Converted: Date
Date: Date
Date_1: Date
Month: Integer
Year: Integer


In [9]:
get_types('OK_well_injection')

OBJECTID: OID
Shape: Geometry
API: String
Well_Name: String
Well_Number: String
Operator_Number: Integer
Report_Date: String
Volume_BPD: Double
Pressure: Double
API_NUM: Integer
Date: Date


## Fix Data Type Issues of Time Components

In [8]:
#Add Date Field in correct dtype for earthquake data
arcpy.AddField_management('OK_earthquakes', 'Date', 'DATE')

#Update values in new row
with arcpy.da.UpdateCursor('OK_earthquakes', ['time', 'Date']) as cursor:
    for row in cursor:
        time_value = row[0]
        if time_value:
            date_value = datetime.strptime(time_value, '%Y-%m-%dT%H:%M:%S.%fZ').date()
            row[1] = date_value
            cursor.updateRow(row)
        else:
            continue

<Result 'G:\\Proj1_SSCI575\\Proj1_SSCI575\\Data Wrangling.gdb\\OK_earthquakes'>

In [6]:
#Add Date Field in correct dtype for well injection data
arcpy.AddField_management('OK_well_injection', 'Date', 'DATE')

#Update values in new row
with arcpy.da.UpdateCursor('OK_well_injection', ['Report_Date', 'Date']) as cursor:
    for row in cursor:
        time_value = row[0]
        if time_value:
            date_value = datetime.strptime(time_value, '%Y-%m-%d %H:%M:%S.%f').date()
            row[1] = date_value
            cursor.updateRow(row)
        else:
            continue

## Display the time component of earthquake and water injection data
- Year 2015 
- `Query & Filter`

In [20]:
#Filter to year 2015 for Earthquake Dataset

#Earthquake dataset - Add Month + Year fields
arcpy.AddField_management('OK_earthquakes', 'Month', 'INTEGER')
arcpy.AddField_management('OK_earthquakes', 'Year', 'INTEGER')

#Add data to created columns
with arcpy.da.UpdateCursor('OK_earthquakes', ['Date', 'Month', 'Year']) as cursor:
    for row in cursor:
        row[1] = int(row[0].month)
        row[2] = int(row[0].year)
        cursor.updateRow(row)

In [9]:
#Query
query = "Year = 2015" 

# Filtered Layer
arcpy.MakeFeatureLayer_management('OK_earthquakes', '2015_Earthquakes', query)

<Result '2015_Earthquakes'>

In [22]:
#Filter to year 2015 for Well Injection Dataset

#Earthquake dataset - Add Month + Year fields
arcpy.AddField_management('OK_well_injection', 'Month', 'INTEGER')
arcpy.AddField_management('OK_well_injection', 'Year', 'INTEGER')

#Add data to created columns
with arcpy.da.UpdateCursor('OK_well_injection', ['Date', 'Month', 'Year']) as cursor:
    for row in cursor:
        row[1] = int(row[0].month)
        row[2] = int(row[0].year)
        cursor.updateRow(row)

In [31]:
#Query
query = "Year <= 2015" 

# Filtered Layer
arcpy.MakeFeatureLayer_management('OK_well_injection', '2012_2015_Well_Injection', query)

<Result '2012_2015_Well_Injection'>

## Locate Null Values

In [40]:
def null_count(feature_class):
    dic = {field_name: 0 for field_name in display_fields(feature_class)}

    #Hashmap Null count to Dictionary
    for idx in range(len(display_fields(feature_class))):
        num = 0
        with arcpy.da.SearchCursor(feature_class, display_fields(feature_class)[idx]) as cursor:
            for row in cursor:
                if row[0] == None:
                    num +=1
                    #print(num)
            dic[display_fields(feature_class)[idx]] = num
            
    #Initialize DF
    null_earthquake = pd.DataFrame(list(dic.items()), columns = ['Field', 'Null_Count'])
    return null_earthquake.sort_values(by='Null_Count', ascending=False).reset_index().iloc[:,1:]

null_count('OK_earthquakes')

,Field,Null_Count
0,Date_1,6544
1,dmin,6361
2,nst,6116
3,magError,6090
4,magNst,5985
5,horizontalError,1505
6,gap,616
7,depthError,425
8,rms,401
9,longitude,1


In [41]:
null_count('OK_well_injection')

,Field,Null_Count
0,OBJECTID,0
1,Shape,0
2,API,0
3,Well_Name,0
4,Well_Number,0
5,Operator_Number,0
6,Report_Date,0
7,Volume_BPD,0
8,Pressure,0
9,API_NUM,0


#### As Null Values can be interpreted as a 0 value - we check these instances too

In [46]:
def zero_count(feature_class):
    dic = {field_name: 0 for field_name in display_fields(feature_class)}

    #Hashmap Null count to Dictionary
    for idx in range(len(display_fields(feature_class))):
        num = 0
        with arcpy.da.SearchCursor(feature_class, display_fields(feature_class)[idx]) as cursor:
            for row in cursor:
                if row[0] == 0:
                    num +=1
                    #print(num)
            dic[display_fields(feature_class)[idx]] = num
            
    #Initialize DF
    null_earthquake = pd.DataFrame(list(dic.items()), columns = ['Field', 'Null_Count'])
    return null_earthquake.sort_values(by='Null_Count', ascending=False).reset_index().iloc[:,1:]

zero_count('OK_well_injection')

,Field,Null_Count
0,Pressure,463385
1,Volume_BPD,53030
2,OBJECTID,0
3,Shape,0
4,API,0
5,Well_Name,0
6,Well_Number,0
7,Operator_Number,0
8,Report_Date,0
9,API_NUM,0


## Impute Null Values
- `Earthquake Data - MagError`: Compute **_Linear Regression_** on basis of 'depthError', 'mag', 'depth', 'magNst' to predict N/A **MagError** --> .026 MAE Testing 

- `Well Injection Data - Pressure` --> Impute w/ **Station Mean Value for given month**
- `Well Injection Data 0 Volume_BPD`  --> Impute w/ **Station Mean Value**

### `Earthquake Data - MagError`

In [12]:
#Retrieve values to Regress - Earthquakes
mag_error_ls, depth_err_ls, mag_ls, depth_ls, mag_nst_ls = [], [], [], [], []
with arcpy.da.SearchCursor('OK_earthquakes', ['magError', 'depthError', 'mag', 'depth', 'magNst']) as cursor:
    for row in cursor:
        if row[0]:
            mag_error_ls.append(row[0])
            if row[1] == None:
                depth_err_ls.append(0)
            else:    
                depth_err_ls.append(row[1])
            mag_ls.append(row[2])
            depth_ls.append(row[3])
            mag_nst_ls.append(row[4])

#Save to Perform LR
regress_df = pd.DataFrame(data=list(zip(mag_error_ls, depth_err_ls, mag_ls, depth_ls, mag_nst_ls)), 
                         columns = ['Mag_error', 'Depth_error', 'mag', 'depth', 'mag_nst'])

#LR Regression
reg_df = pd.read_excel('Regress_df.xlsx')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

#Replace N/A w/ Zero for LR
reg_df['mag_nst'] = reg_df['mag_nst'].apply(lambda i: 0 if pd.isna(i) else i)

#Split
x_train, x_test, y_train, y_test = train_test_split(reg_df[['Depth_error', 'mag', 'depth', 'mag_nst']], 
                                                    reg_df['Mag_error'], test_size=.2, random_state=0)

lr_model = LinearRegression().fit(x_train, y_train) #Fit Model

#Evaluate Accuracy
print('Training Accuracy:', mean_absolute_error(y_train, lr_model.predict(x_train))) #ME - Train
print('Testing Accuracy:', mean_absolute_error(y_test, lr_model.predict(x_test))) #ME - Test

In [12]:
#Fill NA Values w/ LR Model Predictions
Intercept = 0.26097579055728315
b1_depth_error = -0.00207291
b1_mag = -0.04513556
b1_depth = 0.0006205
b1_mag_nst = -0.00066137

with arcpy.da.UpdateCursor('OK_earthquakes', ['magError', 'depthError', 'mag', 'depth', 'magNst']) as cursor:
    for row in cursor:
        try: #Conditional for model execution as 2 other fields are missing vlues
            if row[0] == None and row[1] == None and row[4] == None:
                row[0] = Intercept + b1_mag*row[2] + b1_depth*row[3]

            if row[0] == None and row[4] == None:
                row[0] = Intercept + b1_depth_error*row[1] + b1_mag*row[2] + b1_depth*row[3] 
                cursor.updateRow(row)

            if row[0] == None:
                row[0] = Intercept + b1_depth_error*row[1] + b1_mag*row[2] + b1_depth*row[3] + b1_mag_nst*row[4]
                cursor.updateRow(row)
        except:
            row[0] = 0
            cursor.updateRow(row)        

In [14]:
with arcpy.da.UpdateCursor('OK_earthquakes', ['magError']) as cursor: #replace remaining w/ 0 
    for row in cursor:
        if row[0] == None:
            row[0] = 0
            cursor.updateRow(row)

### `Well Injection Data - Pressure`

In [29]:
#Gather data to agg
well_ls = []
with arcpy.da.SearchCursor('OK_well_injection', ['Month', 'Well_Name', 'Pressure']) as cursor:
    for row in cursor:
        well_ls.append((row[0], row[1].strip(), row[2]))
        

In [30]:
#Group aggregate stats
well_df = pd.DataFrame(data = well_ls, columns = ['Month', 'Well_Name', 'Pressure'])
grouped_wells = well_df.groupby(['Month', 'Well_Name']).agg({'Pressure':'mean'}).reset_index()

In [32]:
#Update Column w/ station avg mean for given month through mapping method
cnt = 0 
with arcpy.da.UpdateCursor('OK_well_injection', ['Month', 'Well_Name', 'Pressure']) as cursor:
    for row in cursor:
        print(cnt)
        if row[2] == 0:
            row[2] = grouped_wells.query(f'Month == {row[0]} & Well_Name == "{row[1].strip()}"')['Pressure'].values[0]
            cnt+=1
        else: continue
        

## `Volume_BPD`

In [37]:
year_ls, name_ls, vol_ls = [], [], []

with arcpy.da.SearchCursor('OK_well_injection', ['Year', 'Well_Name', 'Volume_BPD']) as cursor:
    for row in cursor:
        year_ls.append(row[0])
        name_ls.append(row[1])
        vol_ls.append(row[2])

In [39]:
volume_df = pd.DataFrame(data=list(zip(year_ls,name_ls,vol_ls)), columns=['Year', 'Well_Name', 'Volume_BPD'])

In [40]:
vol_grouped = volume_df.groupby('Well_Name').agg({'Volume_BPD' : 'mean'}).reset_index()
vol_dic = {k:v for k,v in zip(vol_grouped['Well_Name'],vol_grouped['Volume_BPD'] )} #Hashmap

In [45]:
with arcpy.da.UpdateCursor('OK_well_injection', ['Well_Name', 'Volume_BPD']) as cursor: #update
    for row in cursor:
        if row[1] == 0:
            row[1] = vol_dic[row[0]]
            cursor.updateRow(row)

## Distribution of Earthquake and Water Injection Data (Histograms)

In [4]:
#Earthquake Data
eq_yr, eq_month = [], []
with arcpy.da.SearchCursor('OK_earthquakes', ['Year', 'Month']) as cursor:
    for row in cursor:
        eq_yr.append(row[0])
        eq_month.append(row[1])
        
#Injection Well Data
well_yr, well_month = [], []
with arcpy.da.SearchCursor('OK_well_injection', ['Year', 'Month']) as cursor:
    for row in cursor:
        well_yr.append(row[0])
        well_month.append(row[1])

#Initialize Data Frames
earth_df = pd.DataFrame(data = list(zip(eq_yr, eq_month)), columns = ['Year', 'Month'])
well_df = pd.DataFrame(data = list(zip(well_yr, well_month)), columns = ['Year', 'Month'])

In [6]:
#Monthly Distribution Wells -  Aggregate Statistics
well_agg = well_df.reset_index().groupby(['Month']).agg({'index': 'count'}).rename(columns={'index':'count'}).\
reset_index()

well_agg.to_excel('G:\Wells_agg.xlsx')

earth_agg = earth_df.reset_index().groupby(['Month']).agg({'index': 'count'}).rename(columns={'index':'count'}).\
reset_index()

earth_agg.to_excel('G:\earth_agg.xlsx')


In [7]:
well_agg_yr = well_df.reset_index().groupby(['Year']).agg({'index': 'count'}).rename(columns={'index':'count'}).\
reset_index()
well_agg_yr.to_excel('G:\Wells_agg_yr.xlsx')

In [9]:
earth_agg_yr = earth_df.reset_index().groupby(['Year']).agg({'index': 'count'}).rename(columns={'index':'count'}).\
reset_index()
earth_agg_yr.to_excel('G:\Earth_agg_yr.xlsx')

In [ ]:
# # Code to Visualize

# well_agg = pd.read_excel('Wells_agg.xlsx')
# plt.figure(figsize=(8,6))
# sns.barplot(data=well_agg, x='Month', y='count', color = 'skyblue')
# plt.title('Monthly Distribution - Well Injections')
# plt.show()

# earth_agg = pd.read_excel('earth_agg.xlsx')
# plt.figure(figsize=(8,6))
# sns.barplot(data=earth_agg, x='Month', y='count', color = 'red')
# plt.title('Monthly Distribution - Earthquakes')
# plt.show()

# wells_agg_yr = pd.read_excel('Wells_agg_yr.xlsx')
# plt.figure(figsize=(8,6))
# sns.barplot(data=wells_agg_yr, x='Year', y='count', color = 'skyblue')
# plt.title('Monthly Distribution - Well Injections')
# plt.show()

# earth_agg_yr = pd.read_excel('Earth_agg_yr.xlsx')
# plt.figure(figsize=(8,6))
# sns.barplot(data=earth_agg_yr, x='Year', y='count', color = 'red')
# plt.title('Monthly Distribution - Earthquakes')
# plt.show()

In [52]:
#Volume_BPD Histogram
well_yr, well_month, vol_ls = [], [], []
with arcpy.da.SearchCursor('OK_well_injection', ['Year', 'Month', 'Volume_BPD']) as cursor:
    for row in cursor:
        well_yr.append(row[0])
        well_month.append(row[1])
        vol_ls.append(row[2])
well_df = pd.DataFrame(data = list(zip(well_yr, well_month, vol_ls)), columns = ['Year', 'Month', 'Volume_BPD'])

In [56]:
vol_df = well_df.groupby(['Year']).agg({'Volume_BPD' : 'sum'}).reset_index()
vol_df.to_excel('G:\Vol_agg_yr.xlsx')

In [57]:
vol_df = well_df.groupby(['Month']).agg({'Volume_BPD' : 'sum'}).reset_index()
vol_df.to_excel('G:\Vol_agg_month.xlsx')

## Space-Time-Cube

In [9]:
arcpy.AddField_management('OK_earthquakes1', 'Earthquake', 'Integer')#add field to count earthquakes

<Result 'OK_earthquakes1'>

In [10]:
cursor = arcpy.UpdateCursor('OK_earthquakes1')
for row in cursor:
    row.setValue('Earthquake', 1) #add counter
    cursor.updateRow(row)

del cursor

In [11]:
arcpy.AddField_management('OK_Well_injection', 'Injection', 'Integer') #add field to count Well Injections

<Result 'OK_well_injection'>

In [12]:
cursor = arcpy.UpdateCursor('OK_Well_injection')
for row in cursor:
    row.setValue('Injection', 1) #add counter
    cursor.updateRow(row)

del cursor

## Data Clocks

In [27]:
#Data Clocks for Earthquake Magnitude and Water Injection Rates
chart = arcpy.charts.DataClock(dateField="Date", numberField = 'mag', aggregation="MEAN", 
                               timeUnitsRingWedge="YEARS_MONTHS", dataSource='OK_earthquakes1', 
                              title = 'Average Magnitude (2010-2016)') 

chart.exportToSVG("data_clock.svg", width=750, height=500) #Earthquake Data Clock

In [13]:
well_chart = arcpy.charts.DataClock(dateField="Date", numberField = 'INJECTION', aggregation="SUM", 
                               timeUnitsRingWedge="YEARS_MONTHS", dataSource = arcpy.env.workspace + '\\''OK_well_injection',
                                   title = 'Injection Rate (2012-2018)') #Well Injection Data Clock


well_chart.exportToSVG("G:\Proj1_SSCI575\Proj1_SSCI575\data_clock_InjectionRate.svg", width=750, height=500)

In [51]:
volume_chart= arcpy.charts.DataClock(dateField="Date", numberField = 'Volume_BPD', aggregation="SUM", 
                               timeUnitsRingWedge="YEARS_MONTHS", dataSource = arcpy.env.workspace + '\\''OK_well_injection',
                                   title = 'Volume Injection Rate (2012-2018)') #Well Injection Data Clock
volume_chart.exportToSVG("G:\Proj1_SSCI575\Proj1_SSCI575\data_clock_VolumeInjectionRate.svg", width=750, height=500)

## Time-Series Earthquake Magnitude and Water Injection Rate - Medford

In [7]:
#Make new Column w/ City Name 
arcpy.AddField_management('OK_earthquakes', 'City', 'String')

<Result 'OK_earthquakes'>

In [21]:
cursor = arcpy.UpdateCursor('OK_earthquakes', ['place', 'City'])

# Loop through the rows in the cursor
for row in cursor:
    try:
        row.setValue('City', row.getValue('place').split(' ')[3].split(',')[0]) #Add City
        cursor.updateRow(row)
    except:
        row.setValue('City', 'Oklahoma')
        cursor.updateRow(row)

In [24]:
arcpy.SelectLayerByAttribute_management("OK_earthquakes", "SUBSET_SELECTION", "City = 'Medford'")

<Result 'OK_earthquakes_Layer2'>

In [23]:
arcpy.SelectLayerByAttribute_management("OK_earthquakes", "CLEAR_SELECTION")

<Result 'OK_earthquakes_Layer1'>

In [25]:
arcpy.CopyFeatures_management("OK_earthquakes", "Medford_earthquakes")
arcpy.RefreshActiveView()
mxd.save()

<Result 'G:\\Proj1_SSCI575\\Proj1_SSCI575\\Data Wrangling.gdb\\Medford_earthquakes'>

In [27]:
#buffer 5 mile distance of medford
bufferOutput = "Medford_25mi"
bufferDist = "25 miles"
arcpy.analysis.Buffer('Medford_earthquakes1', bufferOutput, bufferDist)

<Result 'G:\\Proj1_SSCI575\\Proj1_SSCI575\\Data Wrangling.gdb\\Medford_25mi'>

In [28]:
#Clip Well Injection to General Medford Area
clipInput = "OK_well_injection"
clipOutput = "Medford_well_injection"
arcpy.analysis.Clip(clipInput, 'Medford_25mi', clipOutput)

<Result 'G:\\Proj1_SSCI575\\Proj1_SSCI575\\Data Wrangling.gdb\\Medford_well_injection'>

In [3]:
#Earthquake Medford agg
earth_date, mag_ls, = [], []

cursor = arcpy.SearchCursor('Medford_earthquakes1', ['updated_Converted', 'mag'])

for row in cursor:
    earth_date.append(row.getValue('updated_Converted'))
    mag_ls.append(row.getValue('mag'))

del cursor

In [4]:
#DF + Time-Series Analysis -> Medford Earthquakes
earth_medford_df = pd.DataFrame(data=list(zip(earth_date, mag_ls)), columns = ['Date', 'Magnitude'])
earth_medford_df

,Date,Magnitude
0,2016-09-24 22:47:15,3.1
1,2016-09-17 01:27:53,2.8
2,2016-09-17 01:27:50,3.4
3,2016-09-17 01:27:46,2.5
4,2016-09-17 01:27:46,2.8
...,...,...
1058,2017-07-14 19:33:25,2.9
1059,2014-11-07 01:51:08,2.5
1060,2014-11-07 01:51:08,2.6
1061,2014-11-07 01:50:53,2.5


In [5]:
#Year and month columns
earth_medford_df['Year'], earth_medford_df['Month'] = [i.year for i in earth_medford_df.Date],[i.month for i in earth_medford_df.Date]

In [6]:
earth_medford_df.to_excel('G:\Proj1_SSCI575\Proj1_SSCI575\medford_earthquakes.xlsx')

In [44]:
agg_df = earth_medford_df.groupby(['Year', 'Month']).agg({'Magnitude' : 'mean'}).reset_index()
agg_df['Date'] = agg_df['Month'].astype(str) + '-' + agg_df['Year'].astype(str)
agg_df['Date'] = agg_df['Date'].astype(str)
agg_df.to_excel('G:\Proj1_SSCI575\Proj1_SSCI575\earth_agg.xlsx')

In [45]:
#Create Viz - Earthquakes Medford
import matplotlib.pyplot as plt
df = pd.read_excel('earth_agg.xlsx')
plt.figure(figsize=(15, 6))
plt.plot(df['Date'], df['Magnitude'], marker='o', linestyle='-', color='red')

#Rotate
plt.xticks(rotation=45)

num_ticks = 15  
total_items = len(df['Date'])
step = max(1, total_items // num_ticks)
xticks_positions = range(0, total_items, step)
xticks_labels = df['Date'][::step]

# Set the x-axis tick positions and labels
plt.xticks(xticks_positions, xticks_labels)

#Labels
plt.xlabel('Date')
plt.ylabel('Avg Magnitude')
plt.title('Medford Earthquake Magnitude (2013-2017)')
plt.show()

In [58]:
#Well Medford agg
date_ls, inject_ls, vol_ls = [], [], []
cursor = arcpy.SearchCursor('Medford_well_injection', ['Date', 'Injection', 'Volume_BPD'])

for row in cursor:
    date_ls.append(row.getValue('Date'))
    inject_ls.append(row.getValue('Injection'))
    vol_ls.append(row.getValue('Volume_BPD'))

del cursor

In [63]:
#DF + Time-Series Analysis -> Medford Wells
well_medford_df = pd.DataFrame(data=list(zip(date_ls, inject_ls, vol_ls)), columns = ['Date', 'Injection', 'Volume_BPD'])
well_medford_df.head(3)

,Date,Injection,Volume_BPD
0,2018-07-07 00:00:00.131072,1,2551.0
1,2018-07-06 00:00:00.065536,1,1283.0
2,2018-07-05 00:00:00.131072,1,634.0


In [64]:
#Year+Month
well_medford_df['Year'], well_medford_df['Month'] = [i.year for i in well_medford_df.Date],[i.month for i in well_medford_df.Date]

In [61]:
agg_df = well_medford_df.groupby(['Year', 'Month']).agg({'Injection' : 'sum'}).reset_index()
agg_df['Date'] = agg_df['Month'].astype(str) + '-' + agg_df['Year'].astype(str)
agg_df['Date'] = agg_df['Date'].astype(str)
agg_df.to_excel('G:\Proj1_SSCI575\Proj1_SSCI575\well_agg.xlsx')

In [65]:
agg_df = well_medford_df.groupby(['Year', 'Month']).agg({'Volume_BPD' : 'sum'}).reset_index() #VOLUME_BPD
agg_df['Date'] = agg_df['Month'].astype(str) + '-' + agg_df['Year'].astype(str)
agg_df['Date'] = agg_df['Date'].astype(str)
agg_df.to_excel('G:\Proj1_SSCI575\Proj1_SSCI575\well_volume_agg.xlsx')

In [ ]:
import matplotlib.pyplot as plt
df = pd.read_excel('well_agg.xlsx')
plt.figure(figsize=(15, 6))
plt.plot(df['Date'], df['Injection'], marker='o', linestyle='-')

#Rotate
plt.xticks(rotation=45)

num_ticks = 15  
total_items = len(df['Date'])
step = max(1, total_items // num_ticks)
xticks_positions = range(0, total_items, step)
xticks_labels = df['Date'][::step]

# Set the x-axis tick positions and labels
plt.xticks(xticks_positions, xticks_labels)

#Labels
plt.xlabel('Date')
plt.ylabel('# Injections')
plt.title('Medford Injection Rate (2012-2018)')
plt.show()

In [56]:
#Look at Counts of Earthquakes Medford over Time
agg_df = earth_medford_df.groupby(['Year', 'Month']).agg({'Magnitude' : 'count'}).reset_index()
agg_df['Date'] = agg_df['Month'].astype(str) + '-' + agg_df['Year'].astype(str)
agg_df['Date'] = agg_df['Date'].astype(str)
agg_df.to_excel('G:\Proj1_SSCI575\Proj1_SSCI575\earth_agg.xlsx')

In [58]:
#Same Viz as above^^

## Space-Time Trend Analysis

In [6]:
#Well injections
arcpy.stpm.EmergingHotSpotAnalysis('G:\Proj1_SSCI575\Proj1_SSCI575\space_time_well-volume.nc', 
                                   'PRESSURE_SUM_ZEROS', 'hotspot_injection-volume_10mi.shp', 
                                  '10 Miles')

<Result 'G:\\Proj1_SSCI575\\Proj1_SSCI575\\hotspot_injection-volume_10mi.shp'>

In [4]:
arcpy.stpm.EmergingHotSpotAnalysis('G:\Proj1_SSCI575\Proj1_SSCI575\space_time_well-volume.nc', 
                                   'PRESSURE_SUM_ZEROS', 'hotspot_injection-volume.shp', 
                                  '5 Miles', 2)

<Result 'G:\\Proj1_SSCI575\\Proj1_SSCI575\\hotspot_injection-volume.shp'>

In [ ]:
arcpy.stpm.EmergingHotSpotAnalysis('G:\Proj1_SSCI575\Proj1_SSCI575\space_time1.nc', 
                                   'EARTHQUAKE_SUM_ZEROS', 'hotspot_earthquakes_10mi.shp', 
                                  '10 Miles', 2)

In [3]:
arcpy.stpm.EmergingHotSpotAnalysis('G:\Proj1_SSCI575\Proj1_SSCI575\well_volume_bpd.nc', 
                                   'VOLUME_BPD_SUM_ZEROS', 'bpd_10mi.shp', 
                                  '5 Miles', 2)

<Result 'G:\\Proj1_SSCI575\\Proj1_SSCI575\\bpd_10mi.shp'>

## Merge All data for 2013 Census-Tract Level

### `Census + Traffic`

In [3]:
census_list, traffic_list = [],[]
with arcpy.da.SearchCursor('census_traffic', ['NAMELSAD10', 'volume_2017']) as cursor:
    for row in cursor:
        census_list.append(row[0])
        traffic_list.append(row[1])

In [4]:
traffic_by_census_df = pd.DataFrame(data=list(zip(census_list,traffic_list)),columns=['Name','Traffic'])

In [5]:
traffic_mean_df = traffic_by_census_df.groupby('Name').agg({'Traffic':'mean'}).reset_index()

In [6]:
traffic_mean_df.query('Name == "Census Tract 1"')['Traffic'].values[0]

3860.5806451612902

In [7]:
arcpy.AddField_management('census_traffic', 'Avg_Traffic', 'DOUBLE')

<Result 'census_traffic'>

In [16]:
cursor = arcpy.UpdateCursor('census_traffic', ['NAMELSAD10', 'Avg_Traffic'])
for row in cursor:
    name = row.getValue('NAMELSAD10')
    row.setValue('Avg_Traffic', traffic_mean_df.query(f'Name == "{name}"')['Traffic'].values[0])
    cursor.updateRow(row)

In [15]:
traffic_mean_df.query('Name == "Census Tract 7799"')

,Name,Traffic
746,Census Tract 7799,1758.888889


In [19]:
# arcpy.DeleteIdentical_management('census_traffic.shp', 'NAMELSAD10', "KEEP_FIRST") #Delete Duplicates

### `Census + Traffic + Schools`

In [3]:
census_list, schools_list = [],[]
with arcpy.da.SearchCursor('census_traffic_schools', ['NAMELSAD10', 'schname']) as cursor:
    for row in cursor:
        census_list.append(row[0])
        schools_list.append(row[1])

In [4]:
schools_df = pd.DataFrame(data=list(zip(census_list,schools_list)),columns=['Name','Schools'])

In [5]:
agg_df = schools_df.groupby('Name').agg({'Schools':'count'}).reset_index()

In [14]:
arcpy.AddField_management('census_traffic_schools', 'Num_Schools1', 'DOUBLE')

<Result 'census_traffic_schools'>

In [6]:
dic = dict(zip(agg_df['Name'], agg_df['Schools']))

In [19]:
cursor = arcpy.UpdateCursor('census_traffic_schools', ['NAMELSAD10', 'Num_Schools'])
for row in cursor:
    name = row.getValue('NAMELSAD10')
    row.setValue('Num_Schools', dic[name])
    cursor.updateRow(row)

### `Census + Traffic + Schools + Earthquakes`

In [30]:
census_list, mag_list = [],[] #Lists holding column data
with arcpy.da.SearchCursor('census_traffic_schools_earthquakes', ['NAMELSAD10', 'mag']) as cursor:
    for row in cursor:
        census_list.append(row[0])
        mag_list.append(row[1])

In [32]:
mag_df = pd.DataFrame(data=list(zip(census_list,mag_list)),columns=['Name','Mag']) #initialize df

In [43]:
agg_df = mag_df.groupby('Name').agg({'Mag':'mean'}).reset_index() #locate avg magnitude by census name

In [51]:
arcpy.AddField_management('census_traffic_schools_earthquakes', 'Avg_Mag', 'DOUBLE') #add field to hashmap

<Result 'census_traffic_schools_earthquakes'>

In [50]:
dic = dict(zip(agg_df['Name'], agg_df['Mag'])) #hashmap

In [52]:
cursor = arcpy.UpdateCursor('census_traffic_schools_earthquakes', ['NAMELSAD10', 'Avg_Mag'])
for row in cursor:
    name = row.getValue('NAMELSAD10')
    row.setValue('Avg_Mag', dic[name]) #fill values with avg magnitude respecting census name
    cursor.updateRow(row)

In [5]:
#Perform for 2013

input_feature_class = "OK_earthquakes"  

output_feature_class = "OK_earthquakes_2013"


sql_expression = "Year = 2013"

# Use the Select tool to create the new feature class based on the query


print(f"New feature class {output_feature_class} created based on the query.")


New feature class OK_earthquakes_2013 created based on the query.


In [29]:
#aggregate
census_list, mag_list = [],[]
with arcpy.da.SearchCursor('census_traffic_schools_quakes_2013', ['NAMELSAD10', 'mag']) as cursor:
    for row in cursor:
        census_list.append(row[0])
        mag_list.append(row[1])

In [30]:
dic = dict(zip(census_list, mag_list))

In [31]:
schools_df = pd.DataFrame(data=list(zip(census_list,mag_list)),columns=['Name','mag'])

In [32]:
agg_df = schools_df.groupby('Name').agg({'mag':'mean'}).reset_index()

In [34]:
dic = dict(zip(agg_df['Name'], agg_df['mag']))

In [35]:
arcpy.AddField_management('census_traffic_schools_quakes_2013', 'Avg_Mag', 'DOUBLE')

<Result 'census_traffic_schools_quakes_2013'>

In [36]:
cursor = arcpy.UpdateCursor('census_traffic_schools_quakes_2013', ['NAMELSAD10', 'Avg_Mag'])
for row in cursor:
    name = row.getValue('NAMELSAD10')
    row.setValue('Avg_Mag', dic[name])
    cursor.updateRow(row)

In [ ]:
#DROP DUPLICATES

In [39]:
#Validates
ls=[]
with arcpy.da.SearchCursor('census_traffic_schools_quakes_2013', ['NAMELSAD10', 'Avg_Mag']) as cursor:
    for row in cursor:
        ls.append(row[1])

In [43]:
len([x for x in ls if x>=0]) #49 counties

49

# Scale Index

In [10]:
#Vectorize data to locate min+max
pop_ls, traffic_ls, schools_ls, mag_ls = [], [], [], []
with arcpy.da.SearchCursor('census_traffic_schools_quakes_2013', ['P0010001', 'Avg_Traffic', 'Num_Schools', 'Avg_Mag']) as cursor:
    for row in cursor:
        pop_ls.append(row[0])
        traffic_ls.append(row[1])
        schools_ls.append(row[2])
        mag_ls.append(row[3])

In [12]:
#Add Normalized Fields
arcpy.AddField_management('census_traffic_schools_quakes_2013', 'Scaled_Pop', 'DOUBLE')
arcpy.AddField_management('census_traffic_schools_quakes_2013', 'Scaled_Schools', 'DOUBLE')
arcpy.AddField_management('census_traffic_schools_quakes_2013', 'Scaled_Traffic', 'DOUBLE')
arcpy.AddField_management('census_traffic_schools_quakes_2013', 'Scaled_Avg_Mag', 'DOUBLE')

<Result 'census_traffic_schools_quakes_2013'>

In [22]:
#Fill Columns with normalized values
def normalize(data_list, value_field, scaled_field):
    with arcpy.da.UpdateCursor('census_traffic_schools_quakes_2013', [value_field, scaled_field]) as cursor:
        for row in cursor:
            try:
                row[1] = (row[0]-min([x for x in data_list if x>=0]))/max([x for x in data_list if x>=0])
                cursor.updateRow(row)
            except:
                row[1] = 0
                cursor.updateRow(row)

                
normalize(pop_ls, 'P0010001', 'Scaled_Pop')
normalize(traffic_ls, 'Avg_Traffic', 'Scaled_Traffic')
normalize(schools_ls, 'Num_Schools', 'Scaled_Schools')
normalize(mag_ls, 'Avg_Mag', 'Scaled_Avg_Mag')

# Risk Index Score

In [25]:
#Add Field for scoring
arcpy.AddField_management('census_traffic_schools_quakes_2013', 'Risk_Score', 'DOUBLE')

<Result 'census_traffic_schools_quakes_2013'>

In [26]:
#Compute Score
with arcpy.da.UpdateCursor('census_traffic_schools_quakes_2013', ['Risk_Score', 'Scaled_Pop', 'Scaled_Traffic', 'Scaled_Schools', 
                                                                 'Scaled_Avg_Mag']) as cursor:
    for row in cursor:
        row[0] = sum(row[1:])*.25 #Update Index (0-1)
        cursor.updateRow(row)

## Distribution Analysis

In [28]:
#Magnitude
mag_ls = []
cursor = arcpy.SearchCursor('OK_earthquakes', ['mag']) 
for row in cursor:
    mag_ls.append(row.getValue('mag'))

df = pd.DataFrame(data=mag_ls, columns=['Magnitude'])
df.to_excel('G:\Agg_Mag.xlsx')

In [30]:
vol_ls = []
cursor = arcpy.UpdateCursor('OK_well_injection', ['Volume_BPD'])
for row in cursor:
    vol_ls.append(row.getValue('Volume_BPD'))
df = pd.DataFrame(data=vol_ls, columns=['Volume_BPD'])
df.to_excel('G:\Agg_Well_volume.xlsx')